In [1]:
# %pip install bs4
# %pip install selenium
# %pip install undetected_chromedriver
# %pip install lxml
# %pip install pendulum
# %pip install gspread oauth2client
# %pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

In [2]:
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
import re
from bs4 import BeautifulSoup
from lxml import etree
import pendulum
import operator as op

driver = uc.Chrome()
time.sleep(3)
driver.get('https://www.indeed.com/q-usa-jobs.html')
time.sleep(6)

In [3]:
keyword_filter= "chief"
location_filter = "california"
page_count_filter = "2"
salary_filter = "200000.069 $ year"
start_date_posted_filter = "2024-07-01 00:00:00"
end_date_posted_filter = "2025-01-31 23:59:59"
time.sleep(3)

In [4]:
search_job_keyword = driver.find_element(By.XPATH, '//input[@aria-label="search: Job title, keywords, or company"]')
time.sleep(3)
search_job_keyword.click()
time.sleep(3)
actions = ActionChains(driver)
time.sleep(3)
actions.key_down(Keys.CONTROL).send_keys('A').key_up(Keys.CONTROL).send_keys(Keys.BACK_SPACE).perform()
time.sleep(3)
search_job_keyword.send_keys(keyword_filter)
time.sleep(3)
search_location_keyword = driver.find_element(By.XPATH, '//input[@aria-label="Edit location"]')
time.sleep(3)
search_location_keyword.click()
time.sleep(3)
search_location_keyword.send_keys(location_filter)
time.sleep(3)
search_button = driver.find_element(By.XPATH, '//button[@class="yosegi-InlineWhatWhere-primaryButton"]')
time.sleep(3)
search_button.submit()
time.sleep(6)

In [5]:
def search_job_indeed():
    global search_job_indeed_list, search_job_indeed_final_list, search_job_indeed_final_list_filter
    search_job_indeed_final_list = []
    time.sleep(3)
    while True:
        try:
            job_indeed_frame = driver.find_elements(By.XPATH, '//div[@id="mosaic-provider-jobcards"]/ul/li/div/div/div/div/div/div/table/tbody/tr/td[1]/div[1]/h2/a')
            time.sleep(3)
            search_job_indeed_list = list(set((job_indeed.get_attribute('href') for job_indeed in job_indeed_frame)))
            time.sleep(3)
            job_indeed_frame[-1].location_once_scrolled_into_view
            time.sleep(6)
        except:
            print('No more job')
            time.sleep(3)
        search_job_indeed_final_list.extend(search_job_indeed_list)
        time.sleep(3)
        search_job_indeed_final_list = list(set(search_job_indeed_final_list))
        time.sleep(3)
        if (len(search_job_indeed_final_list) % 15 == 0 and len(search_job_indeed_final_list) // 15 == int(page_count_filter)) or (len(search_job_indeed_final_list) % 15 != 0 and len(search_job_indeed_final_list) // 15 + 1 == int(page_count_filter)):
            break
        try:
            next_page_button = driver.find_elements(By.XPATH, '//nav[@aria-label="pagination"]/ul/li/a')
            time.sleep(3)
            next_page_button[-1].click()
            time.sleep(3)
            yield len(search_job_indeed_final_list)
        except:
            print('No more page')
            time.sleep(3)
            break

generators = search_job_indeed()
for generator in generators:
    print(generator)

15
15
15
15


In [6]:
if len(search_job_indeed_final_list) % 15 == 0:
    if len(search_job_indeed_final_list) == 0:
        search_job_indeed_final_list_filter = []
    elif len(search_job_indeed_final_list) // 15 + 1 < int(page_count_filter):
        search_job_indeed_final_list_filter = search_job_indeed_final_list
    else:
        search_job_indeed_final_list_filter = search_job_indeed_final_list[:int(page_count_filter) * 15]
else:
    if len(search_job_indeed_final_list) // 15 + 1 < int(page_count_filter):
        search_job_indeed_final_list_filter = search_job_indeed_final_list
    elif len(search_job_indeed_final_list) // 15 + 1 == int(page_count_filter):
        search_job_indeed_final_list_filter = search_job_indeed_final_list[:(int(page_count_filter) - 1) * 15 + len(search_job_indeed_final_list) % 15]
    else:
        search_job_indeed_final_list_filter = search_job_indeed_final_list[:int(page_count_filter) * 15]

In [7]:
len(search_job_indeed_final_list_filter)

30

In [8]:
def get_job_indeed_description():
        global job_indeed_description_list
        job_indeed_description_list = []
        for _, job_indeed_url in enumerate(search_job_indeed_final_list_filter):
            driver.get(job_indeed_url)
            time.sleep(6)
            try:
                    company = driver.find_element(By.XPATH, '//div[@data-testid="jobsearch-CompanyInfoContainer"]/div/div/div/div/div[1]/span/a').text
                    time.sleep(3)
                    sub_company = driver.find_element(By.XPATH, '//div[@data-testid="jobsearch-CompanyInfoContainer"]/div/div/div/div/div[2]/span').text
                    time.sleep(3)
                    if len(re.findall(r'\b\d+\.\d+\b', sub_company)) > 0:
                        total_company = company
                    else:
                        total_company = company + ' ' + sub_company
                    time.sleep(3)
            except:
                    try:
                            time.sleep(3)
                            total_company = driver.find_element(By.XPATH, '//div[@data-testid="jobsearch-CompanyInfoContainer"]/div/div/div/div/div[1]/span/a').text
                            time.sleep(3)
                    except:
                            try:
                                    time.sleep(3)
                                    total_company = driver.find_element(By.XPATH, '//div[@data-testid="jobsearch-CompanyInfoContainer"]/div/div/div/div/div[1]/span').text
                                    time.sleep(3)
                            except:
                                    try:
                                            time.sleep(3)
                                            total_company = driver.find_element(By.XPATH, '//div[starts-with(@class, "jobsearch-InfoHeaderContainer")]/div/div/h2').text
                                            time.sleep(3)
                                    except:
                                            try:
                                                    time.sleep(3)
                                                    total_company = driver.find_element(By.XPATH, '//div[starts-with(@class, "jobsearch-InfoHeaderContainer")]/div/div/h2').text
                                                    time.sleep(3)
                                            except:
                                                    time.sleep(3)
                                                    total_company = ''
                                                    time.sleep(3)
            try:
                    header = driver.find_element(By.XPATH, '//div[starts-with(@class,"jobsearch-JobInfoHeader-title-container")]/h1/span').text
                    time.sleep(3)
            except:
                    try:
                            time.sleep(3)
                            header = driver.find_element(By.XPATH, '//div[starts-with(@class, "jobsearch-InfoHeaderContainer")]/div/div/div/span').text
                            time.sleep(3)
                    except:
                            time.sleep(3)
                            header = ''
                            time.sleep(3)
            try:
                    location = driver.find_element(By.XPATH, '//div[@data-testid="jobsearch-CompanyInfoContainer"]/div/div/div/div[2]/div').text
                    time.sleep(3)
            except:
                    try:
                            time.sleep(3)
                            location = driver.find_element(By.XPATH, '//div[starts-with(@class, "jobsearch-InfoHeaderContainer")]/div/div/div/div[2]/div').text
                            time.sleep(3)
                    except:
                            try:
                                time.sleep(3)
                                location = driver.find_element(By.XPATH, '//div[starts-with(@class, "jobsearch-InfoHeaderContainer")]/div/div[2]/div/div/div/div[2]/div').text
                                time.sleep(3)
                            except:
                                    time.sleep(3)
                                    location = ''
                                    time.sleep(3)
            try:
                soup = BeautifulSoup(driver.page_source, 'lxml')
                time.sleep(3)
                script_context = soup.find_all('script', string=re.compile(r'@context'))[0].getText().strip()
                time.sleep(3)
                standard_date_posted = re.search(r'\"datePosted\":\"([^"]+)\"', script_context).group(1)
                time.sleep(3)
                readable_date_posted = pendulum.parse(standard_date_posted, tz='Asia/Ho_Chi_Minh').to_datetime_string()
                time.sleep(3)
            except:
                try:
                    soup = BeautifulSoup(driver.page_source, 'lxml')
                    time.sleep(3)
                    script_initial_data = soup.find_all('script', string=re.compile(r'window._initialData'))[0].getText().strip()
                    time.sleep(3)
                    standard_date_posted = re.search(r'\"datePublished\":(\d+)', script_initial_data).group(1)
                    time.sleep(3)
                    readable_date_posted = pendulum.from_timestamp(int(standard_date_posted) / 1000, tz='Asia/Ho_Chi_Minh').to_datetime_string()
                    time.sleep(3)
                except:
                        time.sleep(3)
                        readable_date_posted = ''
                        time.sleep(3)
            try:
                soup = BeautifulSoup(driver.page_source, 'lxml')
                time.sleep(3)
                script_initial_data = soup.find_all('script', string=re.compile(r'window._initialData'))[0].getText().strip()
                time.sleep(3)
                salary = re.search(r'\"salaryText\":\"([^"]+)\"', script_initial_data).group(1)
                time.sleep(3)
            except:
                    time.sleep(3)
                    salary = ''
                    time.sleep(3)
            job_indeed_description_list.append((job_indeed_url, total_company, header, salary, location, readable_date_posted))
            yield total_company, header, location, readable_date_posted, salary
generators = get_job_indeed_description()
for generator in generators:
    print(generator)

('Cisco Systems', 'Chief of Staff', 'San Jose, CA', '2025-01-15 01:04:18', '')
('Deep 6 AI', 'Chief Operating Officer', 'Pasadena, CA', '2024-11-12 05:10:08', '$220,000 - $260,000 a year')
('City of Manhattan Beach, CA', 'Fire Chief', '400 15th Street, Manhattan Beach, CA 90266', '2025-01-01 18:54:10', '$247,800 - $321,000 a year')
('US Space Systems Command', 'Chief, Information Protection, SLD 30', 'Vandenberg AFB, CA', '2025-01-15 03:44:04', '$145,180 - $195,200 a year')
('ocV!BE', 'Chief, Engineering Operations', 'Anaheim, CA 92806', '2025-01-03 00:26:22', '$225,000 - $255,000 a year')
('Okta', 'Chief Product Officer, Customer Identity', 'San Francisco, CA', '2025-01-15 03:07:28', '$336,000 - $564,000 a year')
('Ironclad, Inc.', 'Chief of Staff & CEO Communications', 'San Francisco, CA', '2024-12-11 17:10:23', '$192,000 - $240,000 a year')
('Aughdem Recruitment', 'Chief Operating Officer', 'Los Angeles, CA', '2024-10-31 08:49:54', '$200,000 a year')
('California State University St

In [9]:
len(job_indeed_description_list)

30

In [22]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2 import service_account
import pandas as pd

credentials = service_account.Credentials.from_service_account_file('credentials.json')
spreadsheet_service = build('sheets', 'v4', credentials=credentials)

SPREADSHEET_ID = '1F75bELM8_nYQvFBVqReVUZYcPGuuELIa4nh9MJoS9Tg'
RANGE_NAME = 'Sheet1'
sheet = spreadsheet_service.spreadsheets()
result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
values = result.get('values', [])
max_cols = max(len(row) for row in values)
values = [row + [''] * (max_cols - len(row)) for row in values]
df = pd.DataFrame(values[1:], columns=values[0])
df = df.fillna('')

In [29]:
cleared_values = {
    "requests": [
        {
            "updateCells": {
                "range": {
                    "sheetId": 0,
                    "startRowIndex": 1,
                    "endRowIndex": None,
                    "startColumnIndex": 0,
                    "endColumnIndex": 6
                },
                "fields": "userEnteredValue"
            }
        }
    ]
}
spreadsheet_service.spreadsheets().batchUpdate(spreadsheetId=SPREADSHEET_ID, body=cleared_values).execute()

{'spreadsheetId': '1F75bELM8_nYQvFBVqReVUZYcPGuuELIa4nh9MJoS9Tg',
 'replies': [{}]}

In [30]:
df1 = pd.DataFrame(job_indeed_description_list, columns=df.columns).__deepcopy__()
df1

,Link,Company,Title,Salary,Location,Date posted
0,https://www.indeed.com/rc/clk?jk=f8d01815f3044...,Cisco Systems,Chief of Staff,,"San Jose, CA",2025-01-15 01:04:18
1,https://www.indeed.com/rc/clk?jk=1e0d20a749623...,Deep 6 AI,Chief Operating Officer,"$220,000 - $260,000 a year","Pasadena, CA",2024-11-12 05:10:08
2,https://www.indeed.com/rc/clk?jk=a5f09914c6e90...,"City of Manhattan Beach, CA",Fire Chief,"$247,800 - $321,000 a year","400 15th Street, Manhattan Beach, CA 90266",2025-01-01 18:54:10
3,https://www.indeed.com/rc/clk?jk=09e904d57389f...,US Space Systems Command,"Chief, Information Protection, SLD 30","$145,180 - $195,200 a year","Vandenberg AFB, CA",2025-01-15 03:44:04
4,https://www.indeed.com/rc/clk?jk=e874027ae8cd8...,ocV!BE,"Chief, Engineering Operations","$225,000 - $255,000 a year","Anaheim, CA 92806",2025-01-03 00:26:22
5,https://www.indeed.com/rc/clk?jk=f58de66fbe3f7...,Okta,"Chief Product Officer, Customer Identity","$336,000 - $564,000 a year","San Francisco, CA",2025-01-15 03:07:28
6,https://www.indeed.com/rc/clk?jk=f30acfd68c8f9...,"Ironclad, Inc.",Chief of Staff & CEO Communications,"$192,000 - $240,000 a year","San Francisco, CA",2024-12-11 17:10:23
7,https://www.indeed.com/rc/clk?jk=b18e30a244da3...,Aughdem Recruitment,Chief Operating Officer,"$200,000 a year","Los Angeles, CA",2024-10-31 08:49:54
8,https://www.indeed.com/rc/clk?jk=82dfcfcbbc9e4...,California State University Stanislaus,Chief of Staff,"$15,833 - $16,667 a month","One University Circle, Turlock, CA 95382",2024-11-28 01:16:12
9,https://www.indeed.com/rc/clk?jk=86b798c7f6746...,Peckham & McKenney,Police Chief,"$201,706 a year","Bell, CA",2025-01-13 21:27:44


In [31]:
def implement_salary_filter():
    global implement_filter_list
    implement_filter_list = []
    salary_filter_tuple = (salary_filter.split(' ')[0], salary_filter.split(' ')[1], salary_filter.split(' ')[2])
    for index, row in df1.iterrows():
        if op.contains(df1.at[index, 'Salary'], salary_filter_tuple[1]) and op.contains(df1.at[index, 'Salary'], salary_filter_tuple[2]):
            if '.' in df1.at[index, 'Salary']:
                range_salary_tuple = tuple(float(salary.replace(',', '')) for salary in re.findall(r"([\d,\.]+)", df1.iloc[index]['Salary']))
            else:
                range_salary_tuple = tuple(int(salary.replace(',', '')) for salary in re.findall(r"([\d,]+)", df1.iloc[index]['Salary']))
            if len(range_salary_tuple) == 2 and (float(salary_filter_tuple[0]) >= range_salary_tuple[0] and float(salary_filter_tuple[0]) <= range_salary_tuple[1]):
                implement_filter_list.append((row[df1.columns[0]], row[df1.columns[1]], row[df1.columns[2]], row[df1.columns[3]], row[df1.columns[4]], row[df1.columns[5]]))
            elif len(range_salary_tuple) == 1 and (float(salary_filter_tuple[0]) <= range_salary_tuple[0] and op.contains(df1.iloc[index]['Salary'].lower(), 'up to')):
                implement_filter_list.append((row[df1.columns[0]], row[df1.columns[1]], row[df1.columns[2]], row[df1.columns[3]], row[df1.columns[4]], row[df1.columns[5]]))
            elif len(range_salary_tuple) == 1 and (float(salary_filter_tuple[0]) >= range_salary_tuple[0] and op.contains(df1.iloc[index]['Salary'].lower(), 'from')):
                implement_filter_list.append((row[df1.columns[0]], row[df1.columns[1]], row[df1.columns[2]], row[df1.columns[3]], row[df1.columns[4]], row[df1.columns[5]]))
            elif len(range_salary_tuple) == 1 and (float(salary_filter_tuple[0]) == range_salary_tuple[0] and (not op.contains(df1.iloc[index]['Salary'].lower(), 'up to') and not op.contains(df1.iloc[index]['Salary'].lower(), 'from'))):
                implement_filter_list.append((row[df1.columns[0]], row[df1.columns[1]], row[df1.columns[2]], row[df1.columns[3]], row[df1.columns[4]], row[df1.columns[5]]))
    yield from implement_filter_list
salary_generators = implement_salary_filter()
for generator in salary_generators:
    print(generator)

('https://www.indeed.com/rc/clk?jk=f30acfd68c8f9497&bb=lt1r_tZQ4zL8J9tAnPg0gYCD-gT-dtaiCtACImk26oS2dj0SSHV3L_2bl3jY8TMXuDD6nUBfBuOYXi6EBGGneNynZ4R-MGnTBcp3CtKkvfwfqdAWlPmsfr6iDWCFuWJe&xkcb=SoAB67M332ahnUzS5x0HbzkdCdPP&fccid=d9496550189c24bd&vjs=3', 'Ironclad, Inc.', 'Chief of Staff & CEO Communications', '$192,000 - $240,000 a year', 'San Francisco, CA', '2024-12-11 17:10:23')
('https://www.indeed.com/rc/clk?jk=747605dc022c591c&bb=6x6qTYUZBORwCOOax6oCTeUzmDfaK-QxfwG6EGsW49LdEnuzmoqiW8LizSHkuiCQ6waw9ZAddretQpxBrGJtq5ag28eIJtSVrjv98USi1cnjktGy_VPWRGgchnViDrfn&xkcb=SoC-67M332aui1ANWb0TbzkdCdPP&fccid=7d28c1874e2d22be&vjs=3', 'Bill.com', 'Chief of Staff', '$197,600 - $247,500 a year', 'San Jose, CA', '2025-01-07 23:07:29')
('https://www.indeed.com/rc/clk?jk=8de63d2a62df25af&bb=lt1r_tZQ4zL8J9tAnPg0gUdPBY8CgrziKbIh8CqLyncBywt74DyqvsFPtT4v3wLBK9GtI0icWDQ2aHEK4Qhg21LFr4GFmj5Uvdg3Qbd71J7ZTg4TioLO-6JnkHr3p4yI&xkcb=SoCP67M332ahnUzS5x0AbzkdCdPP&fccid=db52f1353185d77b&vjs=3', 'University of Californ

In [32]:
df2 = pd.DataFrame(implement_filter_list, columns=df1.columns).__deepcopy__()
implement_filter_list = []

In [33]:
def implement_date_posted_filter():
    for row in df2.itertuples(index=False):
        if pendulum.parse(start_date_posted_filter, tz='Asia/Ho_Chi_Minh') <= pendulum.parse(row._5, tz='Asia/Ho_Chi_Minh') and pendulum.parse(end_date_posted_filter, tz='Asia/Ho_Chi_Minh') >= pendulum.parse(row._5, tz='Asia/Ho_Chi_Minh'):
            implement_filter_list.append((row.Link, row.Company, row.Title, row.Salary, row.Location, row._5))
    yield from implement_filter_list
date_posted_generators = implement_date_posted_filter()
for generator in date_posted_generators:
    print(generator)
implement_filter_list = list(set(implement_filter_list))

('https://www.indeed.com/rc/clk?jk=f30acfd68c8f9497&bb=lt1r_tZQ4zL8J9tAnPg0gYCD-gT-dtaiCtACImk26oS2dj0SSHV3L_2bl3jY8TMXuDD6nUBfBuOYXi6EBGGneNynZ4R-MGnTBcp3CtKkvfwfqdAWlPmsfr6iDWCFuWJe&xkcb=SoAB67M332ahnUzS5x0HbzkdCdPP&fccid=d9496550189c24bd&vjs=3', 'Ironclad, Inc.', 'Chief of Staff & CEO Communications', '$192,000 - $240,000 a year', 'San Francisco, CA', '2024-12-11 17:10:23')
('https://www.indeed.com/rc/clk?jk=747605dc022c591c&bb=6x6qTYUZBORwCOOax6oCTeUzmDfaK-QxfwG6EGsW49LdEnuzmoqiW8LizSHkuiCQ6waw9ZAddretQpxBrGJtq5ag28eIJtSVrjv98USi1cnjktGy_VPWRGgchnViDrfn&xkcb=SoC-67M332aui1ANWb0TbzkdCdPP&fccid=7d28c1874e2d22be&vjs=3', 'Bill.com', 'Chief of Staff', '$197,600 - $247,500 a year', 'San Jose, CA', '2025-01-07 23:07:29')
('https://www.indeed.com/rc/clk?jk=8de63d2a62df25af&bb=lt1r_tZQ4zL8J9tAnPg0gUdPBY8CgrziKbIh8CqLyncBywt74DyqvsFPtT4v3wLBK9GtI0icWDQ2aHEK4Qhg21LFr4GFmj5Uvdg3Qbd71J7ZTg4TioLO-6JnkHr3p4yI&xkcb=SoCP67M332ahnUzS5x0AbzkdCdPP&fccid=db52f1353185d77b&vjs=3', 'University of Californ

In [34]:
df3 = pd.DataFrame(implement_filter_list, columns=df1.columns).__deepcopy__()
updated_values = [df3.columns.tolist()] + df3.values.tolist()
body = {'values': updated_values}
spreadsheet_service.spreadsheets().values().update(
    spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME,
    valueInputOption='RAW', body=body).execute()

{'spreadsheetId': '1F75bELM8_nYQvFBVqReVUZYcPGuuELIa4nh9MJoS9Tg',
 'updatedRange': 'Sheet1!A1:F11',
 'updatedRows': 11,
 'updatedColumns': 6,
 'updatedCells': 66}